###-----------------------

In [575]:
import numpy as np
import random
import turtle
import time
import serial
from turtle import *
from operator import itemgetter
import struct
import csv

vn = turtle.Screen()
#vn.clear()
robo = turtle.Turtle()
robo.hideturtle()
robo.penup()

engel = turtle.Turtle()
yol = [] #gittiği yolun kaydı
#initialize
bitti = False    # True olunca sonlanıyor

#Seçenekler  ************************************
tekrarla = False # True olunca aynı değerlerle kinci turu yapacak :puanları silmeyecek 
rastgele = True  # engelkoyme biçimi False olursa dosyadan (engeller.csv) okuyor
düzgit = True

#sabitler

blok = 1         # 
numpad = 10      # 9 hücreden oluşuyor, hücrelerin adları numpadle aynı,[0]kullanılmıyor
boy = 30         # robotun boyu ve eni 30cm olarak düşünülebilir
maxziyaret = 5  # aynı hücreye 5 kere giderse sonlanıyor

#odanın boyutlarına göre ekran oluşturuyor
yatay = 3       # yanyana olan numpad sayısı herbiri 90 cm olabilir ( 3 demek 270 cm)
dikey = 2  # dikey olarak numpad sayısı  yatay x dikey   (2 demek 180 cm)

#Açılan pencerenin boyutlarını belirliyor
bigkoor = (3*boy) + 1


#sıfır,sıfır sol üst köşede
#0,0 left upper corner
mode('world')
setworldcoordinates(0,bigkoor,bigkoor,0) #Açılan pencerenin boyutlarını belirliyor
koor = 30

#######

#durum  -1 kullanılmıyor
#        0: gidilmedi
#        1: gidildifor j in range(yatay):
#        2: engelli (gidilemez)
ndurum = np.zeros((numpad),dtype='int')
#her hücrenin kendi adıyla girilebilmesi için [0] kullanım dışı (-1)
ndurum[0] = -1

#hücrelerin puanları
#engelliyse 0 
#her hücrenin başlangıç puanı 10
npuan = 100*np.ones((numpad),dtype='int')
#her hücrenin kendi adıyla girilebilmesi için [0] kullanım dışı (-1)
npuan[0] = -1

#her gidişte artan
nziyaret = np.zeros((numpad),dtype='int')

durum = np.zeros((yatay,dikey,numpad),dtype='int') 
puan = np.zeros((yatay,dikey,numpad),dtype='int') 
ziyaret = np.zeros((yatay,dikey,numpad),dtype='int') 


for x in range(yatay):
    for y in range(dikey):
        durum[x,y] = ndurum
        puan[x,y] = npuan 
        ziyaret[x,y] = nziyaret  

In [576]:

currentyön = 0
#hücrem = 0    #current cell
adımsayısı = 0
engelli = 0  #engel sayısı (bitti kontrolünü de etkiliyor)

In [577]:
port,speed = ('/dev/ttyUSB0', '9600')
try:
    arduino = serial.Serial(port,speed,timeout=.2)
    time.sleep(2)
    print("Connection to " + port + " established succesfully!\n")
    arduino.write("Hello from Python!")
    sensor = 0
    connected = True
except Exception as e:
    print(e)
    connected = False

[Errno 2] could not open port /dev/ttyUSB0: [Errno 2] No such file or directory: '/dev/ttyUSB0'


In [578]:
#Arduino sensoründen data okuma
#sorunlu!
def sensorOku():
    global sensor
    if connected:
        if not arduino.is_open:
            arduino.open()
            time.sleep(1)
        #for say in range(3):
        gelendata = arduino.read(size=1)
        time.sleep(1)
        if gelendata:
            sensor = (gelendata)
            time.sleep(1)
            arduino.close()
            return True
        else:
            #print("sensore ulaşılamadı")
            return False

In [579]:
#Arduino'ya data yolla
def datayolla(data):
    if connected:
        if not arduino.is_open:
            arduino.open()
            time.sleep(1)
            arduino.write(struct.pack('>B', data))
            time.sleep(1)


In [580]:
# değişken haline getirilecek
#cell numpad kurallarıyla
def cell(X,Y,number):
    return {
        1: (5+(koor*X),25 + (koor*Y)),
        2: (15+(koor*X),25 + (koor*Y)),
        3: (25+(koor*X),25 + (koor*Y)),
        4: (5+(koor*X),15 + (koor*Y)),
        5: (15+(koor*X),15 + (koor*Y)),
        6: (25+(koor*X),15 + (koor*Y)),
        7: (5+(koor*X),5 + (koor*Y)),
        8: (15+(koor*X),5 + (koor*Y)),
        9: (25+(koor*X),5 + (koor*Y))
    } [number]

    


#durduğu hücreye göre yönünü gideceği hücreye çeviriyor    
def yönbul(hücre):
    return {
        1: [0,270,180,90],
        2: [180,0,270,90],
        3: [180,270,0,90],
        
        4: [90,0,270,180],
        5: [90,180,0,270],
        6: [90,180,270,0],
        
        7: [90,0,180,270],
        8: [90,180,0,270],
        9: [90,180,270,0]
    } [hücre]


def yönlükomşubul(X,Y,current):

    return {
        1: [(X,Y,2,0),(X,Y,4,270),(X-1,Y,3,180),(X,Y+1,7,90)],
        2: [(X,Y,1,180),(X,Y,3,0),(X,Y,5,270),(X,Y+1,8,90)],
        3: [(X,Y,2,180),(X,Y,6,270),(X+1,Y,1,0),(X,Y+1,9,90)],
        
        4: [(X,Y,1,90),(X,Y,5,0),(X,Y,7,270),(X-1,Y,6,180)],
        5: [(X,Y,2,90),(X,Y,4,180),(X,Y,6,0),(X,Y,8,270)],
        6: [(X,Y,3,90),(X,Y,5,180),(X,Y,9,270),(X+1,Y,4,0)],
        
        7: [(X,Y,4,90),(X,Y,8,0),(X-1,Y,9,180),(X,Y-1,1,270)],
        8: [(X,Y,5,90),(X,Y,7,180),(X,Y,9,0),(X,Y-1,2,270)],
        9: [(X,Y,6,90),(X,Y,8,180),(X,Y-1,3,270),(X+1,Y,7,0)]
    } [current]


def komşubul(X,Y,current):

    return {
        1: [(X,Y,2),(X,Y,4),(X-1,Y,3),(X,Y+1,7)],
        2: [(X,Y,1),(X,Y,3),(X,Y,5),(X,Y+1,8)],
        3: [(X,Y,2),(X,Y,6),(X+1,Y,1),(X,Y+1,9)],
        4: [(X,Y,1),(X,Y,5),(X,Y,7),(X-1,Y,6)],
        5: [(X,Y,2),(X,Y,4),(X,Y,6),(X,Y,8)],
        6: [(X,Y,3),(X,Y,5),(X,Y,9),(X+1,Y,4)],
        7: [(X,Y,4),(X,Y,8),(X-1,Y,9),(X,Y-1,1)],
        8: [(X,Y,5),(X,Y,7),(X,Y,9),(X,Y-1,2)],
        9: [(X,Y,6),(X,Y,8),(X,Y-1,3),(X+1,Y,7)]
    } [current]



In [581]:
#directions 
#0 - east 
#90 - south
#180 - west
#270 - north

def dön(yön):
    global currentyön
    
    turtle.setheading(yön)
    robo.setheading(yön)
    
    currentyön =yön
    return

In [582]:
#Alanı çiz
#Draw the board
if not tekrarla:
    robo.reset()

def çiz(x,y):
    robo.penup()
    robo.setpos(x,y)
    robo.pendown()
    robo.forward(boy)
    return


def numpadçiz(X,Y):

    
    dön(0)
    
    çiz(0+(koor*X),0  + (koor*Y))
    çiz(0+(koor*X),10 + (koor*Y))
    çiz(0+(koor*X),20 + (koor*Y))
    çiz(0+(koor*X),30 + (koor*Y))

    dön(270)

    çiz(30+(koor*X),30  + (koor*Y))
    çiz(20+(koor*X),30  + (koor*Y))
    çiz(10+(koor*X),30  + (koor*Y))
    çiz(0+(koor*X),30  + (koor*Y))        
        
    return



In [583]:
# hem dönüyor, hem döndüğü yönü getiriyor
def doğruyönedön(yer,döndönme):
    yeniyön = None
    #x,y,gidilecekYer = yer
    # buradaki asıl amacımız komşubulmaktan ziyade belli bir komşunun 
    #hangi yönde olduğunu bulmak; 
    #o yüzden x-1 leri filan ve >-1000 leri bırakıyoruz.
    
    
    vahşikomşular = yönlükomşubul(X,Y,hücrem)        
    
    for index, item in enumerate(vahşikomşular):
        if item[2]==yer[2]:
            yeniyön = item[3]            
            if döndönme:
                dön(yeniyön)
    
    return yeniyön

In [584]:
def engelkoy(X,Y,sayı):
    global durum,puan,engelli
    
    #random engel üretirken üst üste aynı yere koyarsa bitti kontrolü hatalı oluyor
    if durum[X,Y,sayı]==0:
        engelli +=1
        engel.setpos(cell(X,Y,sayı))
        durum[X,Y,sayı]= 2
        puan[X,Y,sayı] = -1000  
        engel.stamp()

In [585]:
# c currentyön,döneceğiyön, 2: L90, 3:R90, 4:180 derecelik dönüş
def yolayönekle(c,a):
    return {
        (0,270)   : 2,
        (270,180) : 2,
        (180,90)  : 2,
        (90,0)    : 2,
        (0,90)    : 3,
        (90,180)  : 3,
        (180,270) : 3,
        (270,0)   : 3,
        (0,180)   : 4,
        (90,270)  : 4,
        (180,0)   : 4,
        (270,90)  : 4
    } [(c,a)]    

In [586]:
def Git(yer):
    global hücrem,durum,puan,ziyaret,X,Y,bitti,adımsayısı,yol
    if bitti == False:
        yeniyön = None

        #yön savaşları
        eskiyön = currentyön
        yeniyön = doğruyönedön(yer,True)
        if yeniyön!=eskiyön:
            yol.append(yolayönekle(eskiyön,yeniyön))
            #Arduinoya 2,3,4 den birini yolluyor
            datayolla(yolayönekle(eskiyön,yeniyön))
        #Güncelleme
        X,Y,gidilecekyer = yer
        hücrem = gidilecekyer
        ziyaret[yer] += 1
        durum[yer]=1
        puan[yer]-=0.5
               

        adımsayısı += 1
        yol.append(1)
        #Arduinoya 1 yolluyor
        datayolla(1)
        turtle.setpos(cell(X,Y,gidilecekyer))
        

In [587]:
def puanaGöreKomşuSeç():
    global  bitti
    

    

    karmaliste = []
    sortedkarmaliste = []
    komşulistesi = []
    #eksi X,Y oldumu liste dışı olacak   !!!!!!!!!!!!!!!!!!!
    for yer in komşubul(X,Y,hücrem):
        if yer[0]>=0 and yer[1]>=0 and yer[0]<yatay and yer[1]<dikey and puan[yer]>-500:
            komşulistesi.append(yer)
    
    if len(komşulistesi)==0:
        bitti = True
        return
    
        #tek çıkışlı bir noktaya gittikten sonra geri dönmesin ve bir daha gelmesin diye -500    
    if len(komşulistesi)==1:        
        puan[X,Y,hücrem]=-500
        return komşulistesi[0]
        
    for yer in komşulistesi:
        #puanlistesi.append(puan[yer])
        #yönlistesi.append(yönbul(hücrem)[yer[2]])
        c = []
        c.append(yer)
        c.extend([(doğruyönedön(yer,False)),puan[yer]])
        karmaliste.append(c)
    
    #Düz gideceksin, GO
    sortedkarmaliste  = sorted(karmaliste, key=itemgetter(2),reverse=True)   
    enYüksekPuanlıYer = sortedkarmaliste[0][0]   
    
    if düzgit:
        if len(sortedkarmaliste) == 2:
            ikinciYer =sortedkarmaliste[1][0]
            #puanları eşitse:
            if sortedkarmaliste[0][2] == sortedkarmaliste[1][2]:
                #if1.ninyönü...
                if sortedkarmaliste[0][1] == currentyön:
                    return enYüksekPuanlıYer
                #if2.ninyönü...
                if sortedkarmaliste[1][1] == currentyön:
                    return ikinciYer

        #####333333######
        if len(sortedkarmaliste) == 3:
            ikinciYer =sortedkarmaliste[1][0]
            üçüncüYer = sortedkarmaliste[2][0]
            #puanı birinciyle eşitse:
                    #puanları eşitse:
            if sortedkarmaliste[0][2] == sortedkarmaliste[1][2]:
                #if1.ninyönü...
                if sortedkarmaliste[0][1] == currentyön:
                    return enYüksekPuanlıYer
                #if2.ninyönü...
                if sortedkarmaliste[1][1] == currentyön:
                    return ikinciYer
            if sortedkarmaliste[0][2] == sortedkarmaliste[2][2]:
                #if3.ninyönü...
                if sortedkarmaliste[2][1] == currentyön:
                    return üçüncüYer

        #####444444######
        if len(sortedkarmaliste) == 4:
            ikinciYer =sortedkarmaliste[1][0]
            üçüncüYer = sortedkarmaliste[2][0]        
            dördüncüYer = sortedkarmaliste[3][0]

                    #puanları eşitse:
            if sortedkarmaliste[0][2] == sortedkarmaliste[1][2]:
                #if1.ninyönü...
                if sortedkarmaliste[0][1] == currentyön:
                    return enYüksekPuanlıYer
                #if2.ninyönü...
                if sortedkarmaliste[1][1] == currentyön:
                    return ikinciYer
            if sortedkarmaliste[0][2] == sortedkarmaliste[2][2]:
                #if3.ninyönü...
                if sortedkarmaliste[2][1] == currentyön:
                    return üçüncüYer
            if sortedkarmaliste[0][2] == sortedkarmaliste[3][2]:
                #if3.ninyönü...
                if sortedkarmaliste[3][1] == currentyön:
                    return dördüncüYer




    return enYüksekPuanlıYer

In [588]:
def parılda():
    pass

In [589]:
durum[1,1,1]=0

In [590]:
durum

array([[[-1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0]],

       [[-1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0]],

       [[-1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0]]])

In [591]:
def tara():
    global bitti
    while not bitti:
        #parıltı
        #if  (sum(durum.ravel()) - ((engelli)  + ((8)*yatay*dikey))) == 1:
         #   parılda()
        Git(puanaGöreKomşuSeç())
        # bir hücreye maxziyaret sefer gittiyse bitir
        if max(ziyaret.ravel()) >= maxziyaret :
            bitti = True
        # bütün gidilebilecek noktalara gittiyse bitir
        if sum(durum.ravel()) == (engelli)  + ((8)*yatay*dikey):
            bitti = True
        if bitti :
            if connected:
                if arduino.is_open:
                    arduino.close()
                    time.sleep(1)
    return

In [592]:
def hapisHücreKontrolu():
    global durum
    
    for x in range(yatay):
        for y in range(dikey):
            for h in range(1,10):    
                komşulistesi = []
                for yer in komşubul(x,y,h):
                    if yer[0]>=0 and yer[1]>=0 and yer[0]<yatay and yer[1]<dikey and puan[yer]>-500:
                        komşulistesi.append(yer)
                if len(komşulistesi)==0:
                    durum[x,y,h]=1

In [593]:
for yer in komşubul(0,1,1):
    komşulistesi = []
    if yer[0]>=0 and yer[1]>=0 and yer[0]<yatay and yer[1]<dikey and puan[yer]>-500:
        komşulistesi.append(yer)
    if len(komşulistesi)==0:
        durum[0,1,1]=1

In [594]:
#engel şeklini oluşturuyor
engel.penup() #kalem yukarı çıkıyor: yani inene kadar çizmiyor

engel.color("yellow")
engel.shape("circle")
engel.shapesize(3,3)



In [595]:
#başlangıç
if not tekrarla:
    dön(90)
    X=0
    Y=0
    hücrem = 7
    durum[X,Y,hücrem] = 1
    puan[X,Y,hücrem] -= 0.5
    ziyaret[X,Y,hücrem] += 1
    turtle.setpos(cell(0,0,7))
    turtle.speed(1)
    
    
    if rastgele:
        #engel konuyor
        i=0
        for i in range(10):
            a = random.sample(range(3), 1)
            b = random.sample(range(2), 1)
            c = random.sample(range(10), 1)
            x = a[0]
            a = random.sample(range(3), 1)
            y = b[0]
            no = c[0]
            engelkoy(x,y,no)
    else:
        engeller = []
        engeller = np.genfromtxt('engeller.csv',dtype='int', delimiter=',')
        for i in range(len(engeller)):
            x,y,no = engeller[i]
            engelkoy(x,y,no)
    hapisHücreKontrolu()
    turtle.speed(1)    

In [596]:
turtle.color("pink")
turtle.shape("turtle")
turtle.speed(1)
turtle.shapesize(3,2)
#turtle.setpos(5,5)



In [597]:
#main
turtle.clear()
tekrarla =False  # tekrarlamak istiyorsan True yapcan
yol = []
turtle.pendown()
turtle.pencolor("pink")
adımsayısı = 0
if tekrarla:
    turtle.penup()
    maxziyaret = 100
    bitti =False
    for xx  in range(yatay):
        for yy in range(dikey):
            for sayıre in range(10):
                if durum[xx,yy,sayıre]==1:
                    durum[xx,yy,sayıre]=0
    turtle.pencolor("pink")
    dön(90)
    X=0
    Y=0
    hücrem = 7
    durum[X,Y,hücrem] = 1
    puan[X,Y,hücrem] -= 0.5
    ziyaret[X,Y,hücrem] += 1
    turtle.setpos(cell(0,0,7))
    turtle.speed(1)
    turtle.pendown()
tara()
print (adımsayısı)

117


In [598]:
len(yol)

171

### iyileştirme


In [599]:
puan

array([[[   -1,    96,    97, -1000,    97,    97,    98,    97,    97,
            97],
        [   -1,  -500,    97,    98, -1000,    98,    97,  -500, -1000,
            98]],

       [[   -1,    98,    97,    97,    97,    97,    97,    97,    96,
            97],
        [   -1, -1000,    97,    97, -1000,    97,    97,    98,    97,
            97]],

       [[   -1,    98, -1000,    98,    98,    99,    98, -1000,    99,
            99],
        [   -1,    97,    97,    98,    97,    97,    98,    98,    98,
            98]]])

In [600]:
ziyaret

array([[[0, 4, 3, 0, 3, 3, 2, 3, 3, 3],
        [0, 1, 3, 2, 0, 2, 3, 1, 0, 2]],

       [[0, 2, 3, 3, 3, 3, 3, 3, 4, 3],
        [0, 0, 3, 3, 0, 3, 3, 2, 3, 3]],

       [[0, 2, 0, 2, 2, 1, 2, 0, 1, 1],
        [0, 3, 3, 2, 3, 3, 2, 2, 2, 2]]])

In [601]:
durum

array([[[-1,  1,  1,  2,  1,  1,  1,  1,  1,  1],
        [-1,  1,  1,  1,  2,  1,  1,  1,  2,  1]],

       [[-1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
        [-1,  2,  1,  1,  2,  1,  1,  1,  1,  1]],

       [[-1,  1,  2,  1,  1,  1,  1,  2,  1,  1],
        [-1,  1,  1,  1,  1,  1,  1,  1,  1,  1]]])

In [602]:
puan

array([[[   -1,    96,    97, -1000,    97,    97,    98,    97,    97,
            97],
        [   -1,  -500,    97,    98, -1000,    98,    97,  -500, -1000,
            98]],

       [[   -1,    98,    97,    97,    97,    97,    97,    97,    96,
            97],
        [   -1, -1000,    97,    97, -1000,    97,    97,    98,    97,
            97]],

       [[   -1,    98, -1000,    98,    98,    99,    98, -1000,    99,
            99],
        [   -1,    97,    97,    98,    97,    97,    98,    98,    98,
            98]]])